In [ ]:
!pip install pandas_datareader --upgrade
!git clone https://github.com/pranjal0207/stock-trading-using-machine-learning.git

In [ ]:
import pandas_datareader as pdr
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from importlib import reload
from feature_engineering_helper import ma7, ma21, macd, bollinger_bands, momentum

In [ ]:
stock_list = ["RELIANCE.NS", "TCS.NS", "ICICIBANK.NS", "WIPRO.NS", "INFY.NS", "TRIDENT.NS", "BHEL.NS", "ULTRACEMCO.NS", "SBIN.NS", "TATAMOTORS.NS"]

In [ ]:
for stock in stock_list:
  stock_df = pdr.get_data_yahoo(stock)
  stock_df.to_csv('raw_stocks/' + stock.lower().split(".")[0] + '.csv')

In [ ]:
df = pd.read_csv('raw_stocks/' + stock_list[1].lower().split(".")[0] + '.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
print('No missing data') if sum(stock_df.isna().sum()) == 0 else stock_df.isna().sum()

In [ ]:
files = os.listdir('raw_stocks')
stocks = {}
for file in files:
    if(file == ".ipynb_checkpoints"):
      continue
    name = file.lower().split('.')[0]
    stocks[name] = pd.read_csv('raw_stocks/' + file, engine='python')    
    
    # Return Feature
    stocks[name]['Return'] = round(stocks[name]['Close'] / stocks[name]['Open'] - 1, 3)
    # Change Feature
    # Change of the price from previous day, absolute value
    stocks[name]['Change'] = (stocks[name].Close - stocks[name].Close.shift(1)).fillna(0)
    # Date Feature
    stocks[name]['Date'] = pd.to_datetime(stocks[name]['Date'])
    stocks[name].set_index('Date', inplace=True)
    # Volatility Feature
    stocks[name]['Volatility'] = stocks[name].Close.ewm(21).std()
    # Moving Average, 7 days
    stocks[name]['MA7'] = ma7(stocks[name])
    # Moving Average, 21 days
    stocks[name]['MA21'] = ma21(stocks[name])
    # Momentum
    stocks[name]['Momentum'] = momentum(stocks[name].Close, 3)
    # RSI (Relative Strength Index)
    #stocks[name]['RSI'] = rsi(stocks[name])
    # MACD - (Moving Average Convergence/Divergence)
    stocks[name]['MACD'], stocks[name]['Signal'] = macd(stocks[name])
    # Upper Band and Lower Band for Bollinger Bands
    stocks[name]['Upper_band'], stocks[name]['Lower_band'] = bollinger_bands(stocks[name])
    stocks[name].dropna(inplace=True)
    # Saving
    stocks[name].to_csv('stocks/'+name+'.csv')